In [1]:
# Helpers
import numpy as np

# DB 
import psycopg2
from django.conf import settings

# Evaluation
from sklearn.metrics import roc_curve, auc

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
connection = psycopg2.connect(user = settings.DATABASES["default"]["USER"],
                                  password = settings.DATABASES["default"]["PASSWORD"],
                                  host = settings.DATABASES["default"]["HOST"],
                                  port = settings.DATABASES["default"]["PORT"],
                                  database = settings.DATABASES["default"]["NAME"])
connection.autocommit=True
cursor = connection.cursor()

In [18]:
%%time
query = """select experiment_id, sp_test, sp_hat, x, x_test, sk_hat, q_matrix_hat, mu, auc_threshold
from EDM2020_2020_02_19 where method='fdtf' and train_rmse is not null"""
cursor.execute(query)
row = cursor.fetchone()

CPU times: user 568 ms, sys: 48.5 ms, total: 617 ms
Wall time: 42.1 s


In [22]:
# %%time
update_queries = []
total = 0
while row:
    exp_id, sp_test, sp_hat, X, X_test, sk_hat, q_matrix_hat, mu, auc_threshold = row
    sp_test = np.asarray(sp_test)
    sp_hat = np.asarray(sp_hat)
    sk_hat = np.asarray(sk_hat)
    X = np.asarray(X)
    X_test = np.asarray(X_test)
    sp_hat_test = np.zeros((sp_hat.shape[0], sp_hat.shape[1], sp_test.shape[2]))
    attempts = min(sp_test.shape[2], sp_hat.shape[2])
    sp_hat_test[:, :, :attempts] = sp_hat[:, :, :attempts]
    sk_hat_test = np.zeros((sk_hat.shape[0], sk_hat.shape[1], sp_test.shape[2]))
    sk_hat_test[:, :, :attempts] = sk_hat[:, :, :attempts]
    q_matrix_hat = np.asarray(q_matrix_hat)
    
    # Update X_test shape
    attempts_per_question = X.sum(axis=0).sum(axis=1)
    del_questions = [idx for idx,value in enumerate(attempts_per_question) if value < 3]
    X_test = np.delete(X_test, del_questions, axis=1)
    sp_test = np.delete(sp_test, del_questions, axis=1)
    
    # Calculate SK and SP for the next test attempts
    for attempt in range(1, attempts):
        students = np.where(X_test[:,:, attempt] == 1)
        for student in students:
            sk_hat_test[student, :, attempt] = (2*sk_hat_test[student, :, attempt-1]) + \
                                            2*(1-sk_hat_test[student, :, attempt-1])/(1+np.exp(
                                                -mu*np.dot(X_test[student, :, attempt], q_matrix_hat.T))) - 1
            sp_hat_test[student, :, attempt] = np.dot(sk_hat_test[student, :, attempt], q_matrix_hat)
    
    # Get train predicted values
    y = sp_test[np.where(X_test == 1)]
    y_pred = sp_hat_test[np.where(X_test == 1)]
    
    # Binarize prediction
    y_pred_binary = np.where(np.asarray(y_pred) > auc_threshold, 1, 0)
    
    # Calculate accuracy and RMSE
    acc = np.logical_not(np.logical_xor(y, y_pred_binary)).sum()/len(y)
    rmse = np.sqrt(np.power((y-y_pred), 2)).mean()
    
    update = {
        "test_acc": acc,
        "test_rmse": rmse,
        "test_1":  np.where(y == 1)[0].shape[0],
        "test_0":  np.where(y == 0)[0].shape[0],
        "test_pred_1": np.where(y_pred_binary == 1)[0].shape[0],
        "test_pred_0": np.where(y_pred_binary == 0)[0].shape[0],
    }
    
    
    # Write PSQL query
    update_query = "UPDATE EDM2020_2020_02_19 SET "
    update_list = []
    query_values = []
    for key, value in update.items():
        update_list.append(key + "= %s")
        query_values.append(value)
    
    update_query += ", ".join(update_list) + "where experiment_id = %s"
    query_values.append(exp_id)
    query_values = tuple(query_values)
    query = cursor.mogrify(update_query, query_values)
    update_queries.append(query)
    total += 1
    try:
        row = cursor.fetchone()
    except psycopg2.ProgrammingError:
        row = False

In [12]:
for query in update_queries:
    cursor.execute(query)

In [24]:
total

1